In [13]:
import os 
import re 
import pandas as pd
from tqdm import tqdm
from collections import Counter
from utils import get_completion, num_tokens_from_string, compute_token_price

In [2]:
data2evaluate = pd.read_csv("./cc_book1_qa2.tsv", sep = "\t")
data2evaluate.head()

,Articolo di riferimento,Query,Passage
0,Art. 1 Capacità giuridica,1. Quando si acquisisce la capacità giuridica ...,La capacità giuridica si acquista dal momento ...
1,Art. 1 Capacità giuridica,2. A quale condizione sono subordinati i dirit...,La capacità giuridica si acquista dal momento ...
2,Art. 2 Maggiore età. Capacità di agire,1. A quale età si acquisisce la maggiore età s...,La maggiore età è fissata al compimento del di...
3,Art. 2 Maggiore età. Capacità di agire,2. Quali atti può compiere una persona al ragg...,La maggiore età è fissata al compimento del di...
4,Art. 2 Maggiore età. Capacità di agire,3. Esistono leggi speciali che stabiliscono un...,La maggiore età è fissata al compimento del di...


In [14]:
pssg_query_list = data2evaluate[["Query","Passage"]].to_records(index = False).tolist()
pssg_query_list = " ".join([" ".join(i) for i in pssg_query_list])

In [16]:
num_tokens_from_string(pssg_query_list) * 5e-05

11.41245

In [3]:
evaluation_prompt = """###Instruction###
Sei un esperto in giurisprudenza. Di seguito ti verranno mostrati un testo
e una domanda. Il tuo compito è stabilire se la risposta alla domanda è contenuta nel testo.
Puoi utilizzare solo i seguenti due OUTPUT validi: ["SI", "NO"].
L'OUTPUT è "SI" se la la risposta alla domanda è contenuta nel testo.
L'OUTPUT "NO" se la risposta alla domanda non è contenuta nel testo.
Per poter dire "SI" la risposta alla domanda deve essere strettamente e chiaramente nel testo.
Restituisci solamente "SI" o "NO" e null'altro.


###TESTO###
{text}

###DOMANDA###
{query}

OUTPUT: """

In [18]:
(num_tokens_from_string(evaluation_prompt)* data2evaluate.shape[0]) * 5e-06 

0.9644750000000001

In [23]:
re.sub(r"\{query\}", "iaia", re.sub(r"\{text\}", "ioio", evaluation_prompt))

'###Instruction###\nSei un esperto in giurisprudenza. Di seguito ti verranno mostrati un testo\ne una domanda. Il tuo compito è stabilire se la risposta alla domanda è contenuta nel testo.\nPuoi utilizzare solo due label: ["SI", "NO"].\nL\'OUTPUT è "SI" se la la risposta alla domanda è contenuta nel testo.\nL\'OUTPUT "NO" se la risposta alla domanda non è contenuta nel testo.\nPer poter dire "SI" la risposta alla domanda deve essere strettamente e chiaramente nel testo.\nRestituisci solamente "SI" o "NO" e null\'altro.\n\n\n###TESTO###\nioio\n\n###DOMANDA###\niaia\n\nOUTPUT: '

In [4]:
data2evaluate.shape

(1115, 3)

In [ ]:
#trial 
answ = []
for row in tqdm(data2evaluate[:500].itertuples()):
    query = re.sub(" secondo il testo?", "", row.Query[2:]).strip()
    passage = row.Passage.strip()
    
    prompt =  re.sub(r"\{query\}", query, re.sub(r"\{text\}", passage, evaluation_prompt))
    answ.append(get_completion(prompt, "gpt-4o"))


500it [04:20,  1.92it/s]


In [ ]:
Counter(answ)

Counter({'SI': 409, 'NO': 91})

In [56]:
evaluate_df = data2evaluate[:500]
evaluate_df["IsQueryRelevant"] = answ
evaluate_df

C:\Users\user\AppData\Local\Temp\ipykernel_16300\2126568384.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  evaluate_df["IsQueryRelevant"] = answ


,Articolo di riferimento,Query,Passage,IsQueryRelevant
0,Art. 1 Capacità giuridica,1. Quando si acquisisce la capacità giuridica ...,La capacità giuridica si acquista dal momento ...,SI
1,Art. 1 Capacità giuridica,2. A quale condizione sono subordinati i dirit...,La capacità giuridica si acquista dal momento ...,SI
2,Art. 2 Maggiore età. Capacità di agire,1. A quale età si acquisisce la maggiore età s...,La maggiore età è fissata al compimento del di...,SI
3,Art. 2 Maggiore età. Capacità di agire,2. Quali atti può compiere una persona al ragg...,La maggiore età è fissata al compimento del di...,NO
4,Art. 2 Maggiore età. Capacità di agire,3. Esistono leggi speciali che stabiliscono un...,La maggiore età è fissata al compimento del di...,SI
...,...,...,...,...
495,Art. 191 Scioglimento della comunione,2. In che modo può essere deciso lo scioglimen...,La comunione si scioglie per la dichiarazione ...,SI
496,Art. 192 Rimborsi e restituzioni,1. Quali somme devono essere rimborsate alla c...,Ciascuno dei coniugi è tenuto a rimborsare all...,SI
497,Art. 192 Rimborsi e restituzioni,2. In quali circostanze un coniuge è tenuto a ...,Ciascuno dei coniugi è tenuto a rimborsare all...,SI
498,Art. 192 Rimborsi e restituzioni,3. Quali condizioni devono essere soddisfatte ...,Ciascuno dei coniugi è tenuto a rimborsare all...,SI


In [58]:
# evaluate_df.to_csv("evaluation_df_book1_trial500q.tsv", sep = "\t")

----
# Second Half

In [12]:
#trial 
answ = []
for row in tqdm(data2evaluate[500:].itertuples()):
    query = re.sub(" secondo il testo?", "", row.Query[2:]).strip()
    passage = row.Passage.strip()
    
    prompt =  re.sub(r"\{query\}", query, re.sub(r"\{text\}", passage, evaluation_prompt))
    answ.append(get_completion(prompt, "gpt-4o")) 



0it [00:00, ?it/s]

615it [06:07,  1.68it/s]


In [23]:
answ2 = list(map(lambda x : re.sub(r"\"", "", x).strip(), answ))

In [24]:
Counter(answ2)

Counter({'SI': 473, 'NO': 142})

In [35]:
evaluate_df = data2evaluate.iloc[500:, :]
evaluate_df["IsQueryRelevant"] = answ2
evaluate_df

C:\Users\user\AppData\Local\Temp\ipykernel_25304\816839947.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  evaluate_df["IsQueryRelevant"] = answ2


,Articolo di riferimento,Query,Passage,IsQueryRelevant
500,Art. 192 Rimborsi e restituzioni,5. In quale momento si effettuano i rimborsi e...,Ciascuno dei coniugi è tenuto a rimborsare all...,SI
501,Art. 192 Rimborsi e restituzioni,6. In quali circostanze il giudice può autoriz...,Ciascuno dei coniugi è tenuto a rimborsare all...,SI
502,Art. 192 Rimborsi e restituzioni,7. Quali beni può prelevare il coniuge credito...,Ciascuno dei coniugi è tenuto a rimborsare all...,SI
503,Art. 192 Rimborsi e restituzioni,8. Qual è l'ordine di prelievo dei beni comuni...,Ciascuno dei coniugi è tenuto a rimborsare all...,SI
504,Art. 193 Separazione giudiziale dei beni,1. In quali casi può essere pronunziata la sep...,La separazione giudiziale dei beni può essere ...,SI
...,...,...,...,...
1110,Art. 451 Forza probatoria degli atti,3. Che valore hanno le indicazioni estranee al...,"Gli atti dello stato civile fanno prova, fino ...",SI
1111,"Art. 452 Mancanza, distruzione o smarrimento d...",1. Quali sono le condizioni che permettono di ...,Se non si sono tenuti i registri o sono andati...,NO
1112,"Art. 452 Mancanza, distruzione o smarrimento d...","2. Cosa accade se la mancanza, distruzione, al...",Se non si sono tenuti i registri o sono andati...,SI
1113,Art. 453 Annotazioni,1. Quali sono le condizioni che permettono di ...,Nessuna annotazione può essere fatta sopra un ...,SI


In [36]:
first_500_df = pd.read_csv("evaluation_df_book1_trial500q.tsv", sep = "\t", index_col = False).drop("Unnamed: 0", axis = 1)


whole_df = pd.concat([first_500_df, evaluate_df], axis = 0)

whole_df.shape

(1115, 4)

In [37]:
whole_df.to_csv("evaluate_df_book1_complete.tsv", sep = "\t")

In [38]:
Counter(whole_df["IsQueryRelevant"].tolist())

Counter({'SI': 882, 'NO': 233})

In [40]:
882/1115

0.7910313901345292

In [1]:
import pickle
with open("../qa_results/all_dfs.pkl", "rb") as infile:
    dfs = pickle.load(infile)

In [2]:
for d in dfs:
    print(d.shape)

(873, 3)
(1818, 3)
(3918, 3)
(6051, 3)
(6944, 3)


In [16]:
import json
data = json.load(open("../data/cc_books_clean_whole2_nonum.json", "r"))
cc_book = data["cc_libro_II"]
assert len(dfs[0]["Articolo di riferimento"].unique()) == len(cc_book), "Number of unique articles does not match number of inputs"

In [17]:
len(cc_book)

345